In [1]:
import requests
import pandas as pd
import os
from datetime import datetime
import time

In [2]:
def fetch_stock_data(symbol: str, api_key: str) -> pd.DataFrame:
    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": symbol,
        "outputsize": "full",
        "apikey": api_key
    }
    print(f"Fetching data for {symbol}...")
    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        raise Exception(f"API call failed for {symbol} with status code {response.status_code}")

    data = response.json()
    if "Time Series (Daily)" not in data:
        raise Exception(f"Unexpected response format for {symbol}: {data}")

    df = pd.DataFrame.from_dict(data["Time Series (Daily)"], orient="index")

    df.reset_index(inplace=True)
    df.rename(columns={"index": "date"}, inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values("date")
    df = df.astype(float, errors='ignore')  # Convert numeric columns
    df = df.rename(columns={
        "1. open": "open",
        "2. high": "high",
        "3. low": "low",
        "4. close": "close",
        "5. volume": "volume"
    })
    # df.index = pd.to_datetime(df.index)
    # df = df.sort_index()
    # df = df.astype(float)
    return df

def save_data(df: pd.DataFrame, symbol: str):
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    filename = f"{symbol}_historical_data.csv"
    path = os.path.join(OUTPUT_DIR, filename)
    df.to_csv(path)
    print(f"Saved: {path}")

In [3]:
# === CONFIG ===
ALPHA_VANTAGE_API_KEY = "TM7F9WONU1MJI0ZH"  # Store your API key in an environment variable
BASE_URL = "https://www.alphavantage.co/query"
OUTPUT_DIR = "data"
TICKERS = ["AAPL", "GOOGL", "MSFT", "AMZN", "TSLA", "META", "NVDA", "NFLX", "IBM", "INTC"]

# INTERVAL = "daily"  # Can be 'daily', 'weekly', etc.

In [4]:
def main():
    if not ALPHA_VANTAGE_API_KEY:
        raise ValueError("Please set your ALPHA_VANTAGE_API_KEY environment variable.")

    for ticker in TICKERS:
        try:
            df = fetch_stock_data(ticker, ALPHA_VANTAGE_API_KEY)
            df = df.rename({'Unnamed: 0': 'date'}, axis=1)
            save_data(df, ticker)
            time.sleep(15)  # Alpha Vantage free tier: 5 requests per minute
        except Exception as e:
            print(f"Failed to fetch/save {ticker}: {e}")

if __name__ == "__main__":
    main()

Fetching data for AAPL...
Saved: data/AAPL_historical_data.csv
Fetching data for GOOGL...
Saved: data/GOOGL_historical_data.csv
Fetching data for MSFT...
Saved: data/MSFT_historical_data.csv
Fetching data for AMZN...
Saved: data/AMZN_historical_data.csv
Fetching data for TSLA...
Saved: data/TSLA_historical_data.csv
Fetching data for META...
Saved: data/META_historical_data.csv
Fetching data for NVDA...
Saved: data/NVDA_historical_data.csv
Fetching data for NFLX...
Saved: data/NFLX_historical_data.csv
Fetching data for IBM...
Saved: data/IBM_historical_data.csv
Fetching data for INTC...
Saved: data/INTC_historical_data.csv


In [ ]:
def analyze_stock_dates():
    for ticker in TICKERS:
        file_path = os.path.join(OUTPUT_DIR, f"{ticker}_historical_data.csv")

        if not os.path.exists(file_path):
            print(f"File not found for {ticker}")
            continue

        df = pd.read_csv(file_path, parse_dates=True)
        if 'Unnamed: 0' in df.columns:
            df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

        unique_dates = df['date'].nunique()
        print(f"{ticker}: {unique_dates} unique dates")

analyze_stock_dates()

AAPL: 6401 unique dates
GOOGL: 5196 unique dates
MSFT: 6401 unique dates
AMZN: 6401 unique dates
TSLA: 3721 unique dates
META: 3244 unique dates
NVDA: 6401 unique dates
NFLX: 5760 unique dates
IBM: 6401 unique dates
INTC: 6401 unique dates
